<a href="https://colab.research.google.com/github/shinyoungin4137/AI_final_project/blob/main/final_project_finetuning__w_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial intelligence Final Project: Finetuning_BERT

Copyright (C) Computer Science & Engineering, Soongsil University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them (November 2025.)

BERT(Bidirectional Encoder Representations from Transformers) is a groundbreaking model in the NLP domain. This tutorial provides a step-by-step guide on how to fine-tune the lightweight BERT variant using Hugging Face's transformers library for text classification tasks.<br>

This is about BERT (Devlin et al., 2018).<br>
https://arxiv.org/abs/1810.04805

The code below are based on the following link. <br>
https://medium.com/@khang.pham.exxact/text-classification-with-bert-7afaacc5e49b


### Fine-tune the model
1. Design your model's prediction head
2. Finetune the model by changing the hyperparameters.
3. You will get a score based on the your (hidden) test accuracy for text classification (ranking-based).  

### Submitting your work:
<font color=red>**DO NOT clear the final outputs**</font> so that we can grade both your code and results.  


Now proceed to the code.


## Install libraries

In [1]:
import os

In [2]:
!python3 -m pip install pandas
!python3 -m pip install transformers

### import libraries

In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from tqdm import tqdm
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [4]:
import re
import torch.nn as nn

#  Preprocessing
def clean_text_bert(text):
    # 1. Remove HTML Tag
    text = re.sub(r'<.*?>', ' ', text)

    # 2. Remove URL
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # 3. Remove Unnecessary Spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

### Specify your GPU number if necessary

In [5]:
%env CUDA_VISIBLE_DEVICES = 0

if torch.cuda.is_available() is True:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

env: CUDA_VISIBLE_DEVICES=0


## Preparing dataset

In [6]:
from google.colab import drive
import os

# 1. 구글 드라이브 연결 (마운트)
drive.mount('/content/drive')

# 2. 작업 위치를 '내 프로젝트 폴더'로 이동
# (폴더 이름이 'final_project_AI'가 맞는지 꼭 확인하세요!)
project_path = '/content/drive/MyDrive/final_project_AI'

try:
    os.chdir(project_path)
    print(f"✅ 작업 위치 이동 완료! 현재 위치: {os.getcwd()}")

    # 3. 파일이 진짜 있는지 확인사살
    if os.path.exists('./data/IMDB Dataset.csv'):
        print("🎉 파일 찾음! 이제 로드하시면 됩니다.")
    else:
        print("⚠️ 폴더는 왔는데 파일이 안 보여요. 파일명을 확인해보세요.")
        print(f"현재 폴더 파일 목록: {os.listdir('.')}")
        if os.path.exists('data'):
             print(f"data 폴더 내부 목록: {os.listdir('data')}")

except FileNotFoundError:
    print("❌ 폴더 경로가 틀린 것 같습니다. 구글 드라이브 폴더명을 다시 확인해주세요.")

Mounted at /content/drive
✅ 작업 위치 이동 완료! 현재 위치: /content/drive/MyDrive/final_project_AI
🎉 파일 찾음! 이제 로드하시면 됩니다.


link : https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

1. Download the dataset from attached link.
2. Move the downloaded zip file under the "data" directory and then unzip the zip file.
3. Run the following cell

In [7]:
def load_imdb_data(data_file_path):
    if os.path.exists(data_file_path):
        df = pd.read_csv(data_file_path)
        texts = df['review'].tolist()
        labels = [1 if sentiment == "positive" else 0 for sentiment in df['sentiment'].tolist()]
        return texts, labels
    else:
        raise FileNotFoundError(f"The file '{data_file_path}' does not exist.")

data_file_path = './data/IMDB Dataset.csv'
texts, labels = load_imdb_data(data_file_path)

## Dataset class

In [8]:
class CustomTextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_seq_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = clean_text_bert(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

## Classifier head for BERT( Design your model's prediction head )

In [9]:
class CustomBERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(CustomBERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)

        # Add Dropout
        self.dropout = nn.Dropout(p=0.3)
        # Add Classifier
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output

        output = self.dropout(pooled_output)
        logits = self.classifier(output)

        return logits

## train and evaluation method

In [10]:
def train_model(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in tqdm(data_loader, desc="Train"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # Define Loss Function with Label Smoothing
        criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

        # Calculate Loss
        loss = criterion(outputs, labels)

        loss.backward()

        #Gredient Clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

def evaluate_model(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Validation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

## Hyper-parameter settings

In [11]:
# Set up parameters
# Hint: generally, 5 ~ 10 epochs will be enough.
bert_model_name = 'bert-base-uncased'
num_classes = 2

max_seq_length = 512
batch_size = 8
num_epochs = 5
learning_rate = 2e-5

## get data utils

In [12]:
######################## DO NOT CHANGE ########################
train_texts, val_texts, train_labels, val_labels = \
train_test_split(texts, labels, test_size=0.2, random_state=42)
######################## DO NOT CHANGE ########################

tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = CustomTextClassificationDataset(train_texts, train_labels, tokenizer, max_seq_length)
val_dataset = CustomTextClassificationDataset(val_texts, val_labels, tokenizer, max_seq_length)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Define model, optimizer and scheduler

In [13]:
from torch.cuda.amp import autocast, GradScaler
from torch.optim.swa_utils import AveragedModel, SWALR

# 1. 모델 정의
model = CustomBERTClassifier(bert_model_name, num_classes).to(device)

# 2. LLRD (층별 학습률 차등 적용) 함수 정의
def get_optimizer_params(model, base_lr, weight_decay=0.01):
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = []

    # 분류기 헤드 (가장 높은 학습률)
    optimizer_grouped_parameters += [
        {'params': [p for n, p in model.classifier.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay, 'lr': base_lr},
        {'params': [p for n, p in model.classifier.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0, 'lr': base_lr}
    ]

    # BERT 레이어 (아래로 갈수록 학습률 0.95배씩 감소)
    layers = [model.bert.embeddings] + list(model.bert.encoder.layer)
    layers.reverse()
    decay_rate = 0.95
    lr = base_lr
    for layer in layers:
        lr *= decay_rate
        optimizer_grouped_parameters += [
            {'params': [p for n, p in layer.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay, 'lr': lr},
            {'params': [p for n, p in layer.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0, 'lr': lr}
        ]
    return optimizer_grouped_parameters

# 3. 옵티마이저 설정 (기본 5e-5)
optimizer_params = get_optimizer_params(model, base_lr=5e-5)
optimizer = AdamW(optimizer_params)

# 4. 스케줄러 설정 (Warmup 10%)
total_steps = len(train_dataloader) * num_epochs
warmup_steps = int(total_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

# 5. SWA (Stochastic Weight Averaging) 설정
swa_model = AveragedModel(model)
swa_start = int(len(train_dataloader) * (num_epochs * 0.75)) # 75% 시점부터 시작
swa_scheduler = SWALR(optimizer, swa_lr=2e-5)

# 6. FP16 & Gradient Accumulation 설정
scaler = GradScaler()
accumulation_steps = 4 # 배치 사이즈 8 x 4 = 32 효과

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/tmp/ipython-input-2757693311.py:46: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


## Train model and save best model

In [14]:
eval_acc = 0
global_step = 0
model_path = './finetuned_bert.pth'

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # --- Train (FP16 + Accumulation) ---
    model.train()
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    for i, batch in enumerate(tqdm(train_dataloader, desc="Train")):
        global_step += 1

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # FP16 연산 (속도 UP, 메모리 절약)
        with autocast():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels) / accumulation_steps

        # Backward (Scaler 사용)
        scaler.scale(loss).backward()

        # Accumulation Step마다 업데이트
        if (i + 1) % accumulation_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()

            # SWA 적용 시점인지 확인
            if global_step > swa_start:
                swa_model.update_parameters(model)
                swa_scheduler.step()
            else:
                scheduler.step()

            optimizer.zero_grad()

    # --- Validation (SWA or Standard) ---
    # SWA가 켜졌으면 SWA 모델로 평가 (성능 더 좋음)
    eval_target_model = swa_model if global_step > swa_start else model

    # SWA용 BatchNorm 업데이트 (안전장치)
    if global_step > swa_start:
        torch.optim.swa_utils.update_bn(train_dataloader, swa_model, device=device)

    # 평가 실행 (기존 출력 포맷 유지)
    accuracy, report = evaluate_model(eval_target_model, val_dataloader, device)
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(report)

    # 최고 성능 모델 저장
    if eval_acc < accuracy:
        torch.save(eval_target_model.state_dict(), model_path)
        print('Saved Trained Model.')
        eval_acc = accuracy

Epoch 1/5


Train:   0%|          | 0/5000 [00:00<?, ?it/s]/tmp/ipython-input-2279880958.py:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Validation: 100%|██████████| 1250/1250 [01:47<00:00, 11.57it/s]


Validation Accuracy: 0.9304
              precision    recall  f1-score   support

           0       0.94      0.92      0.93      4961
           1       0.92      0.94      0.93      5039

    accuracy                           0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000

Saved Trained Model.
Epoch 2/5


Train:   0%|          | 0/5000 [00:00<?, ?it/s]/tmp/ipython-input-2279880958.py:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Validation: 100%|██████████| 1250/1250 [01:48<00:00, 11.57it/s]


Validation Accuracy: 0.9424
              precision    recall  f1-score   support

           0       0.96      0.93      0.94      4961
           1       0.93      0.96      0.94      5039

    accuracy                           0.94     10000
   macro avg       0.94      0.94      0.94     10000
weighted avg       0.94      0.94      0.94     10000

Saved Trained Model.
Epoch 3/5


...

Validation Accuracy: 0.9344
              precision    recall  f1-score   support

           0       0.97      0.89      0.93      4961
           1       0.90      0.97      0.94      5039

    accuracy                           0.93     10000
   macro avg       0.94      0.93      0.93     10000
weighted avg       0.94      0.93      0.93     10000

Epoch 4/5


Train:   0%|          | 0/5000 [00:00<?, ?it/s]/tmp/ipython-input-2279880958.py:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Validation: 100%|██████████| 1250/1250 [01:47<00:00, 11.60it/s]


Validation Accuracy: 0.9428
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      4961
           1       0.94      0.94      0.94      5039

    accuracy                           0.94     10000
   macro avg       0.94      0.94      0.94     10000
weighted avg       0.94      0.94      0.94     10000

Saved Trained Model.
Epoch 5/5


Train:   0%|          | 0/5000 [00:00<?, ?it/s]/tmp/ipython-input-2279880958.py:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Validation: 100%|██████████| 1250/1250 [01:48<00:00, 11.56it/s]


Validation Accuracy: 0.9462
              precision    recall  f1-score   support

           0       0.95      0.94      0.95      4961
           1       0.95      0.95      0.95      5039

    accuracy                           0.95     10000
   macro avg       0.95      0.95      0.95     10000
weighted avg       0.95      0.95      0.95     10000

Saved Trained Model.
